<a href="https://colab.research.google.com/github/yeialgov/MAYC_ML_Revenue_Prediction/blob/patch-1/Classification%20and%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# download the libraries
import pandas as pd
from pandas import Series,DataFrame
import warnings
from copy import deepcopy
import os
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import random
%matplotlib inline

In [0]:
from sklearn.preprocessing import LabelEncoder

# Getting the data

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
cd '/content/gdrive/My Drive/ReDi_DA_Project/Datasets_Clean/'

/content/gdrive/My Drive/ReDi_DA_Project/Datasets_Clean


In [0]:
all_data = pd.read_csv('all_data.csv')


In [55]:
all_data.isnull().sum().sum()

0

In [8]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109926 entries, 0 to 2109925
Data columns (total 33 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   geoNetwork.subContinent                       object 
 1   customDimension.value                         object 
 2   totals.transactionRevenue                     float64
 3   fullVisitorId                                 object 
 4   totals.bounces                                int64  
 5   geoNetwork.region                             object 
 6   date                                          object 
 7   visitStartTime                                int64  
 8   totals.sessionQualityDim                      int64  
 9   trafficSource.adwordsClickInfo.isVideoAd      bool   
 10  totals.timeOnSite                             int64  
 11  device.deviceCategory                         object 
 12  geoNetwork.city                               object 
 1

In [0]:
'''
columns to hotcode

channelGrouping                               object  unique values:  8
date                                          int64  - convert to date time
fullVisitorId                                 object - organisers insist to be left as str
device.browser                                object - unique values:  129
device.operatingSystem                        object - unique values:  24
device.isMobile                               bool   - unique values:  2
device.deviceCategory                         object - unique values:  3
geoNetwork.continent                          object - unique values:  6
geoNetwork.subContinent                       object - unique values:  23
geoNetwork.country                            object - unique values:  228
geoNetwork.region                             object - unique values:  284
geoNetwork.metro                              object - unique values:  123
geoNetwork.city                               object - unique values:  956
geoNetwork.networkDomain                      object - unique values:  41982
totals.sessionQualityDim                      object - convert to interger
totals.timeOnSite                             object - convert to interger
trafficSource.source                          object - unique values:  345
trafficSource.medium                          object - unique values:  7
trafficSource.isTrueDirect                    bool   - unique values:  2 
trafficSource.adwordsClickInfo.adNetworkType  object - unique values:  3
trafficSource.adwordsClickInfo.isVideoAd      bool   - unique values:  2 
customDimension.value                         object - unique values:  5
'''

# Model to search the visitors who made transactions

## Function to define Target Column

In [0]:
def is_transaction(n):
  if n == 0    :    return 0
  else         :    return 1

all_data['is_transaction']=all_data['totals.transactions'].apply(is_transaction)

In [10]:
all_data['is_transaction'].value_counts()


0    2085062
1      24864
Name: is_transaction, dtype: int64

## Encoding Categorical Values

In [0]:
TARGET = 'is_transaction'
CAT_COLS = all_data.columns[all_data.dtypes == object].tolist()
CONT_COLS = [c for c in all_data.columns if c not in CAT_COLS and c != TARGET]


In [0]:
CAT_COLS.remove('date')
CAT_COLS.remove('fullVisitorId')
CAT_COLS

In [25]:
CONT_COLS.remove('totals.transactions')
CONT_COLS.remove('totals.transactionRevenue')
CONT_COLS.remove('totals.totalTransactionRevenue')
CONT_COLS

['geoNetwork.subContinent',
 'customDimension.value',
 'totals.bounces',
 'visitStartTime',
 'totals.sessionQualityDim',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'totals.timeOnSite',
 'visitNumber',
 'totals.hits',
 'totals.newVisits',
 'trafficSource.isTrueDirect',
 'device.isMobile',
 'visitId',
 'totals.pageviews']

In [34]:
M1_data = deepcopy(all_data[CAT_COLS + CONT_COLS + [TARGET]])

M1_data[CAT_COLS].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109926 entries, 0 to 2109925
Data columns (total 15 columns):
 #   Column                                        Dtype 
---  ------                                        ----- 
 0   fullVisitorId                                 object
 1   geoNetwork.region                             object
 2   device.deviceCategory                         object
 3   geoNetwork.city                               object
 4   trafficSource.source                          object
 5   device.operatingSystem                        object
 6   geoNetwork.metro                              object
 7   trafficSource.adwordsClickInfo.adNetworkType  object
 8   trafficSource.medium                          object
 9   channelGrouping                               object
 10  geoNetwork.continent                          object
 11  geoNetwork.country                            object
 12  device.browser                                object
 13  dataset     

In [0]:
#leb = LabelEncoder()
for col in CAT_COLS:
    M1_data[col] = LabelEncoder().fit_transform(M1_data[col])

## Split Train and Test

In [41]:
M1_data['dataset'].value_counts()

1    1708337
0     401589
Name: dataset, dtype: int64

In [0]:
train = M1_data[M1_data['dataset']==1]
test = M1_data[M1_data['dataset']==0]
train.drop('dataset',axis=1,inplace=True)
test.drop('dataset',axis=1, inplace=True)

In [0]:
CAT_COLS.remove('dataset')

In [47]:
train.shape, test.shape

((1708337, 29), (401589, 29))

## Model

In [0]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
rmse = lambda true, pred: mse(true, pred) ** 0.5

In [0]:
x_train = train[CAT_COLS + CONT_COLS]
x_test = test[CAT_COLS + CONT_COLS]
y_train = train[[TARGET]]
y_test = test[[TARGET]]

In [1]:
#train.shape

NameError: ignored

In [49]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1708337, 27), (401589, 27), (1708337, 1), (401589, 1))

In [0]:
model = RandomForestClassifier( 
    n_estimators=50, # Hyperparameter 1
    max_depth=2,      # Hyperparameter 2
    random_state=0
)

In [0]:
trained_model = model.fit(x_train, y_train)

In [0]:
def get_error(pred, true):
    print(f'RMSE:       {rmse(true, pred)}')
    print(f'MAE:        {mae(true, pred)}')
    print(f'r2:         {r2(true, pred)}')
    print(f'accuracy:   {accuracy(true, pred)}')

In [56]:
get_error(pred = trained_model.predict(x_test),
          true = y_test)

RMSE:       0.12530016451982023
MAE:        0.015700131228694014
r2:         -0.015950557067829774
accuracy:   0.984299868771306
